In [48]:
import folium
import geopandas as gpd
import pandas as pd
import requests
import leafmap.foliumap as leafmap
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from mapclassify import classify
from folium.plugins import GroupedLayerControl


In [49]:
# Nice
url_communes = 'https://www.data.gouv.fr/api/1/datasets/r/7a861138-93c9-44ba-9c9a-2805e9b39b70'
url_pav = 'http://opendata.nicecotedazur.org/data/storage/f/2020-09-02T13%3A44%3A50.928Z/pav.geojson'
url_population_france = 'https://object.files.data.gouv.fr/hydra-parquet/hydra-parquet/630e7917-02db-4838-8856-09235719551c.parquet'

# Préparation des données géographiques des communes

In [50]:
try:
    communes_nice = gpd.read_file('../data/raw/gdf_communes_Nice.shp')
except:
    file = requests.get(url_communes)
    data = file.content
    communes_nice = gpd.read_file(data)
    communes_nice.to_file('../data/raw/gdf_communes_Nice.shp', index = False)

In [51]:
communes_nice.head()

,NOM,CODE_INSEE,geometry
0,LE BROC,06025,"POLYGON ((7.15198 43.84555, 7.15223 43.84554, ..."
1,BONSON,06021,"POLYGON ((7.19215 43.87625, 7.19211 43.87614, ..."
2,GILETTE,06066,"POLYGON ((7.13509 43.86372, 7.13519 43.86362, ..."
3,GATTIERES,06064,"POLYGON ((7.13402 43.79003, 7.1343 43.78975, 7..."
4,ASPREMONT,06006,"POLYGON ((7.24337 43.80202, 7.24351 43.80193, ..."


In [52]:
# uniformisation des noms de communes
communes_nice['NOM'] = communes_nice['NOM'].apply(lambda x : x.replace('-', ' '))

# Préparation des données sur les pav

In [53]:
# Récupération des data sur les pav
try:
    pav_nice = gpd.read_file('../data/raw/gdf_pav_Nice.shp')
except:
    file = requests.get(url_pav)
    data = file.content
    pav_nice = gpd.read_file(data)
    pav_nice.to_file('../data/raw/gdf_pav_Nice.shp', index = False)

In [54]:
pav_nice.head()

,COMMUNE,JOUR,ADRESSE,CONTENEUR,ID_NCA,TYPE,geometry
0,Nice,Lundi,RUE SORGENTINO / RUE MARECHAL VAUBAN,VERRE,48088V,AERIEN,POINT (7.29189 43.71094)
1,Nice,Lundi,CHEMIN DU MONT GROS / BD BISCHOFFSHEIM (DANS C...,VERRE,49088V,AERIEN,POINT (7.29798 43.7174)
2,Nice,Lundi,BOULEVARD ARMEE DES ALPES FACE AU N°22,VERRE,50088V,AERIEN,POINT (7.29272 43.70805)
3,Nice,Lundi,BD ARMEE DES ALPES / RUE LAURENT GIAUME,VERRE,51088V,AERIEN,POINT (7.29227 43.70668)
4,Nice,Lundi,BOULEVARD PIERRE SOLA ANGLE RUE BEAUTRUCH,VERRE,52088V,AERIEN,POINT (7.28954 43.70601)


In [55]:
# Modification du nom de variable pour uniformisation avec les données d'autres villes
pav_nice = pav_nice.rename({'CONTENEUR': 'Type de flux'}, axis = 1)

In [56]:
# erreur de saisie sur un identifiant : identifiant dupliqué alors que pav différents
# on modifie l'identifiant pour que le pav soit bien compté
pav_nice.loc[pav_nice['ID_NCA'].duplicated()]
pav_nice.loc[pav_nice['ID_NCA'] == '50027T']
pav_nice.loc[2071, 'ID_NCA'] = '50027T_2'

In [57]:
# uniformisation des noms de communes
pav_nice['COMMUNE'] = pav_nice['COMMUNE'].apply(lambda x : str(x).upper())
pav_nice['COMMUNE'] = pav_nice['COMMUNE'].apply(lambda x : x.replace('É', 'E'))
mapping = {
    'TOURETTE LEVENS' : 'TOURRETTE LEVENS',
    'BEAULIEU' : 'BEAULIEU SUR MER',
    'SAINT ANDRE' : 'SAINT ANDRE DE LA ROCHE',
    'LA ROQUETTE' : 'LA ROQUETTE SUR VAR',
    'SAINT SAUVEUR' : 'SAINT SAUVEUR SUR TINEE',
    'SAINT DALMAS' : 'SAINT DALMAS LE SELVAGE',
}

pav_nice['COMMUNE'] = pav_nice['COMMUNE'].replace(mapping)


In [58]:
# calcul du nombre de pav par commune et par type
nb_pav_communes = pav_nice.groupby(['COMMUNE', 'Type de flux']).count()['ID_NCA'].reset_index()
nb_pav_communes

,COMMUNE,Type de flux,ID_NCA
0,ASPREMONT,EMBALLAGES MENAGERS,8
1,ASPREMONT,ORDURES MENAGERES,3
2,ASPREMONT,PAPIER,2
3,ASPREMONT,VERRE,5
4,BAIROLS,EMBALLAGES MENAGERS,1
...,...,...,...
185,VENCE,VERRE,50
186,VILLEFRANCHE SUR MER,EMBALLAGES MENAGERS,13
187,VILLEFRANCHE SUR MER,ORDURES MENAGERES,1
188,VILLEFRANCHE SUR MER,PAPIER,16


In [59]:
# ajout des lignes pour lesquelles la combinaison commune/types de flux n'existe pas
# afin que ces quartiers apparaissent tout de même sur la carte, avec une densité de 0

types_flux = pav_nice['Type de flux'].unique()

for commune in nb_pav_communes['COMMUNE'].unique():
    commune_df = nb_pav_communes.loc[nb_pav_communes['COMMUNE'] == commune]
    for type in types_flux:
        if type not in commune_df['Type de flux'].unique():
            new_line = pd.DataFrame.from_dict(
                {
                    'COMMUNE' : [commune],
                    'Type de flux' : [type],
                    'ID_NCA' : [0]
                }
            )
            nb_pav_communes = pd.concat([nb_pav_communes, new_line])

# Préparation des données de population

In [60]:
# Récupération des données de recensement en France métropolitaine
try:
    population_france = pd.read_parquet('../data/raw/population_france.parquet')
except:
    population_france = pd.read_parquet(url_population_france)
    population_france.to_parquet('../data/raw/population_france.parquet', index = False)

In [61]:
population_france.head()

,objectid,reg,dep,cv,codgeo,libgeo,p13_pop,p14_pop,p15_pop,p16_pop,p17_pop,p18_pop,p19_pop,p20_pop,p21_pop
0,115658,52,85,8502,85062,Châteauneuf,968,993,1013,1027,1056,1085,1114,1118,1134
1,115659,26,58,5808,58300,Urzy,1839,1835,1828,1802,1775,1749,1746,1747,1742
2,115660,43,70,7012,70137,Chassey-lès-Montbozon,218,217,216,215,217,215,215,220,225
3,115661,21,51,5123,51649,Vitry-le-François,13174,13144,12805,12552,12133,11743,11376,11458,11454
4,115662,11,78,7811,78638,Vaux-sur-Seine,4749,4715,4788,4857,4927,4929,5010,5020,5083


In [62]:
# extracion des données les plus récentes
population_21 = population_france[['codgeo', 'libgeo', 'p21_pop']]

# extraction et jointure des données de la métropole de Nice
communes_nice_21 = communes_nice.merge(population_21, left_on = 'CODE_INSEE', right_on = 'codgeo', how = 'left')
communes_nice_21 = communes_nice_21.drop(['codgeo', 'libgeo'], axis = 1)

# Traitement des dataframes

## Jointure des autres dataframe

In [63]:
communes_nice_21.head(1)

,NOM,CODE_INSEE,geometry,p21_pop
0,LE BROC,06025,"POLYGON ((7.15198 43.84555, 7.15223 43.84554, ...",1422


In [64]:
nb_pav_communes.head(1)

,COMMUNE,Type de flux,ID_NCA
0,ASPREMONT,EMBALLAGES MENAGERS,8


In [65]:
# merge des données de carte, de population et de nombre de points d'apport
nice = (communes_nice_21
             .merge(nb_pav_communes, left_on = 'NOM', right_on = 'COMMUNE', how = 'left')
             .rename({'ID_NCA' : 'nb_points'}, axis = 1)
)

nice['nb_points'] = nice['nb_points'].fillna(0).astype(int)

nice

,NOM,CODE_INSEE,geometry,p21_pop,COMMUNE,Type de flux,nb_points
0,LE BROC,06025,"POLYGON ((7.15198 43.84555, 7.15223 43.84554, ...",1422,LE BROC,EMBALLAGES MENAGERS,3
1,LE BROC,06025,"POLYGON ((7.15198 43.84555, 7.15223 43.84554, ...",1422,LE BROC,PAPIER,7
2,LE BROC,06025,"POLYGON ((7.15198 43.84555, 7.15223 43.84554, ...",1422,LE BROC,TEXTILE,2
3,LE BROC,06025,"POLYGON ((7.15198 43.84555, 7.15223 43.84554, ...",1422,LE BROC,VERRE,9
4,LE BROC,06025,"POLYGON ((7.15198 43.84555, 7.15223 43.84554, ...",1422,LE BROC,ORDURES MENAGERES,0
...,...,...,...,...,...,...,...
242,TOURNEFORT,06146,"POLYGON ((7.14422 43.97222, 7.14517 43.97209, ...",149,TOURNEFORT,EMBALLAGES MENAGERS,3
243,TOURNEFORT,06146,"POLYGON ((7.14422 43.97222, 7.14517 43.97209, ...",149,TOURNEFORT,PAPIER,3
244,TOURNEFORT,06146,"POLYGON ((7.14422 43.97222, 7.14517 43.97209, ...",149,TOURNEFORT,TEXTILE,1
245,TOURNEFORT,06146,"POLYGON ((7.14422 43.97222, 7.14517 43.97209, ...",149,TOURNEFORT,VERRE,5


# On passe en Geodataframe pour les cartes

In [66]:
#Conversion de nice en geodataframe
gdf_nice = gpd.GeoDataFrame(nice, geometry='geometry').to_crs(3857)

In [67]:
# Calcul de la densité des points d'apport par quartier
gdf_nice['pav_pour_1000_hab'] = gdf_nice['nb_points']/gdf_nice['p21_pop']*1000

In [68]:
# ajout de classes d'intervalles pour uniformiser les couleurs
# choix des classes d'intervalles
taux_max = max(gdf_nice['pav_pour_1000_hab'])
ref_bins = [0.0, 0.5, 1, 2, 3, 4, 5, taux_max]
ref_labels = ['< 0,5', '(0.5, 1]', '(1, 2]', '(2, 3]', '(3, 4]', '(4, 5]', '> 5']

# création d'un dictionnaire de couleurs pour chaque classe d'intervalle à partir de la cmap 'Oranges'
n_bins = len(ref_bins)-1
cmap = mpl.colormaps['Greens']
ref_colors = cmap(np.linspace(0, 1, n_bins))
custom_cmap = mpl.colors.ListedColormap(ref_colors)


color_dict = {}
color_dict[np.nan] = '#808080'
for n in range(n_bins):
    color_dict[ref_labels[n]] = mpl.colors.rgb2hex(custom_cmap(n))

In [69]:
gdf_nice['label'] = pd.cut(gdf_nice['pav_pour_1000_hab'], bins = ref_bins, labels = ref_labels)
gdf_nice['color'] = gdf_nice['label'].map(color_dict)
gdf_nice = gdf_nice.rename({'p21_pop' : 'population'}, axis = 1).drop('NOM', axis = 1)
gdf_nice

,CODE_INSEE,geometry,population,COMMUNE,Type de flux,nb_points,pav_pour_1000_hab,label,color
0,06025,"POLYGON ((796154.935 5441571.416, 796182.414 5...",1422,LE BROC,EMBALLAGES MENAGERS,3,2.109705,"(2, 3]",#73c476
1,06025,"POLYGON ((796154.935 5441571.416, 796182.414 5...",1422,LE BROC,PAPIER,7,4.922644,"(4, 5]",#0b7734
2,06025,"POLYGON ((796154.935 5441571.416, 796182.414 5...",1422,LE BROC,TEXTILE,2,1.40647,"(1, 2]",#aedea7
3,06025,"POLYGON ((796154.935 5441571.416, 796182.414 5...",1422,LE BROC,VERRE,9,6.329114,> 5,#00441b
4,06025,"POLYGON ((796154.935 5441571.416, 796182.414 5...",1422,LE BROC,ORDURES MENAGERES,0,0.0,NaN,#808080
...,...,...,...,...,...,...,...,...,...
242,06146,"POLYGON ((795290.58 5461144.367, 795396.282 54...",149,TOURNEFORT,EMBALLAGES MENAGERS,3,20.134228,> 5,#00441b
243,06146,"POLYGON ((795290.58 5461144.367, 795396.282 54...",149,TOURNEFORT,PAPIER,3,20.134228,> 5,#00441b
244,06146,"POLYGON ((795290.58 5461144.367, 795396.282 54...",149,TOURNEFORT,TEXTILE,1,6.711409,> 5,#00441b
245,06146,"POLYGON ((795290.58 5461144.367, 795396.282 54...",149,TOURNEFORT,VERRE,5,33.557047,> 5,#00441b


## Carte du nombre de point d'apport

In [70]:
legend_colors = color_dict
if np.nan in legend_colors.keys():
    legend_colors.pop(np.nan)

In [ ]:
# fond de carte
map = leafmap.Map(
    location = [43.95, 7.22],
    tiles = 'cartodbpositron',
    zoomSnap = 0.1,
    zoom_start = 10.3,
    max_zoom=17,
    control_scale=True
    )


# ajout des couches pour chaque type de flux
layers = {}

for type in types_flux:
    layer_name = 'fg_'+ type 
    label = type
    layers[layer_name] = folium.FeatureGroup(name = label).add_to(map)

    
    pav_nice.loc[pav_nice['Type de flux'] == type].explore(
        m = layers[layer_name],
        marker_kwds=dict(radius=2, fill=True),
        color = 'Red'
    )


    gdf_nice.loc[gdf_nice['Type de flux'] == type].explore(
    m = layers[layer_name], 
    color = 'color',
    tooltip = ['COMMUNE', 'population', 'nb_points', 'pav_pour_1000_hab'],
    popup = True,
    legend = False,
    style_kwds = dict(
            fillOpacity = 0.9,
            color = 'green',
            opacity = 0.5),
        highlight_kwds = dict(
            fillOpacity = 1)
            )



    map.add_child(layers[layer_name])





# Contrôle de l'affichage des couches
GroupedLayerControl(
    groups={'Type de flux': [layer for layer in layers.values()]},
    collapsed=False,
).add_to(map)




# ajout de la légende
legend_style = {
    "position": "fixed",
    "z-index": "9999",
    "border": "2px solid grey",
    "background-color": "rgba(255, 255, 255, 0.8)",
    "border-radius": "10px",
    "padding": "5px",
    "font-size": "14px",
    "bottom": "20px",
    "right": "5px",
}

map.add_legend(
    title="nb pav/1000 hab.", legend_dict=legend_colors, draggable=False, style=legend_style
)


# ajout du titre
map_title = "Nombre de pav pour 1000 hab - NICE"
title_html = f'<h1 align="center" style="font-size:24px" >{map_title}</h1>'
map.get_root().html.add_child(folium.Element(title_html))


map.save('../cartes/map_densite_pav_pop_Nice.html')

